In [1]:
import tensorflow as tf

In [2]:
mnist = tf.keras.datasets.mnist

In [4]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize the data
x_train, x_test = x_train / 255., x_test / 255.

11501568/11490434 [==============================] - 3s 0us/step


In [6]:
x_train, y_train, x_test, y_test = x_train[:1000], y_train[:1000], x_test[:1000], y_test[:1000]

In [7]:
x_train.shape

(1000, 28, 28)

In [8]:
x_train = tf.expand_dims(x_train.astype("float32"), axis=-1)
x_test = tf.expand_dims(x_test.astype("float32"), axis=-1)

In [9]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(1000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [10]:
x_train.shape

TensorShape([1000, 28, 28, 1])

In [11]:
# Write the model
class MyModel(tf.keras.Model):
    
    def __init__(self, loss, optimizer):
        super().__init__()
        self.conv1 = tf.keras.layers.Conv2D(32, (3, 3), activation="relu")
        self.flatten = tf.keras.layers.Flatten()
        self.d1 = tf.keras.layers.Dense(128, activation="relu")
        self.d2 = tf.keras.layers.Dense(10)
        
        self.loss = loss
        self.optimizer = optimizer
        
    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)
    
    # fit step 
    def train_step(self, images, labels):
        with tf.GradientTape() as tape: 
            predictions = self.call(images)
            loss = self.loss(labels, predictions)
        gradients = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        return predictions, loss, gradients
    
    def test_step(self, images, labels):
        predictions = self.call(images)
        loss = self.loss(labels, predictions)
        return predictions, loss
    
    

In [13]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

In [14]:
my_model = MyModel(loss=loss_object, optimizer=optimizer)

In [16]:
train_loss = tf.keras.metrics.Mean()
test_loss = tf.keras.metrics.Mean()

train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

In [20]:
EPOCHS = 5

writer = tf.summary.create_file_writer('summary/run1')

for epoch in range(EPOCHS):
    # Reset the metrics at the start of the epoch
    train_loss.reset_states()
    test_loss.reset_states()
    train_accuracy.reset_states()
    test_accuracy.reset_states()
    
    for train_images, train_labels in train_ds:
        pred_train, loss_train, grads_train = my_model.train_step(train_images, train_labels)
        train_loss(loss_train)
        train_accuracy(train_labels, pred_train)
        
    for test_images, test_labels in test_ds:
        pred_test, loss_test = my_model.test_step(test_images, test_labels)
        test_loss(loss_test)
        test_accuracy(test_labels, pred_test)
        
    with writer.as_default():
        tf.summary.scalar('Train loss', train_loss.result(), step=epoch)
        tf.summary.scalar('Test loss', test_loss.result(), step=epoch)
        tf.summary.scalar('Train accuracy', train_accuracy.result(), step=epoch)
        tf.summary.scalar('Test accuracy', test_accuracy.result(), step=epoch)
        
    message = (
        f'Epoch {epoch + 1}, Train loss: {train_loss.result()}, '
        f'Test loss: {test_loss.result()}, '
        f'Train accuracy: {train_accuracy.result()}, '
        f'Test accuracy: {test_accuracy.result()}'
    )
    print(message)

Epoch 1, Train loss: 0.0015438738046213984, Test loss: 0.4732486307621002, Train accuracy: 1.0, Test accuracy: 0.8840000033378601
Epoch 2, Train loss: 0.0012891648802906275, Test loss: 0.48155179619789124, Train accuracy: 1.0, Test accuracy: 0.8840000033378601
Epoch 3, Train loss: 0.001057018293067813, Test loss: 0.4852418601512909, Train accuracy: 1.0, Test accuracy: 0.8809999823570251
Epoch 4, Train loss: 0.000890868017449975, Test loss: 0.4898553490638733, Train accuracy: 1.0, Test accuracy: 0.8849999904632568
Epoch 5, Train loss: 0.000672508031129837, Test loss: 0.49300500750541687, Train accuracy: 1.0, Test accuracy: 0.8870000243186951


### Get Access to the tensorboard
$ tensorboard --logdir=summary